In [51]:
import os
import numpy as np, pandas as pd, seaborn as sns,matplotlib.pyplot as plt, plotly.express as px

In [69]:
import datetime

In [33]:
pd.set_option('display.max_rows',500)
sns.set(rc={'figure.figsize':(10,6)})

In [71]:
today = datetime.datetime.today()

In [20]:
sunspot = pd.read_csv('Datasets/sunspots-SN_d_tot_V2.0.csv',delimiter=';',names=['year','month','day','fractional_date','daily_sunspots','stDev','#observations','definitive'])

In [21]:
#-1 - missing value for Daily Sunspots
#For Definitive/Provisional indicator, '1' indicates that the value is definitive. '0' indicates that the value is still provisional.

In [59]:
sunspot.dtypes

year                        int64
month                       int64
day                         int64
fractional_date           float64
daily_sunspots              int64
stDev                     float64
#observations               int64
definitive                  int64
date               datetime64[ns]
dtype: object

In [63]:
sunspot.tail()

,year,month,day,fractional_date,daily_sunspots,stDev,#observations,definitive,date
73895,2020,4,26,2020.318,14,1.0,23,0,2020-04-26
73896,2020,4,27,2020.321,29,2.0,20,0,2020-04-27
73897,2020,4,28,2020.324,15,2.6,18,0,2020-04-28
73898,2020,4,29,2020.327,24,1.6,22,0,2020-04-29
73899,2020,4,30,2020.329,20,1.2,20,0,2020-04-30


In [24]:
sunspot.definitive.value_counts()

1    73779
0      121
Name: definitive, dtype: int64

In [64]:
sunspot['date'] = pd.to_datetime((sunspot['year'].astype(str)+sunspot['month'].astype(str)+sunspot['day'].astype(str)),format='%Y%m%d',errors='coerce')

In [72]:
sunspot = sunspot.loc[sunspot['date']<=today] # remove some incorrect dates

In [73]:
fig = px.scatter(sunspot, x='date', y='daily_sunspots',width=1200, height=600)
fig.show()

In [45]:
sunspot['date'].tail()

73895    2050
73896    2051
73897    2052
73898    2053
73899    2054
Name: date, dtype: int64

In [67]:
today

<bound method Timestamp.today of <class 'pandas._libs.tslibs.timestamps.Timestamp'>>